# Imports

In [4]:
from modules import *
from mlp_numpy import MLP
import argparse
import numpy as np
import os
from tqdm.auto import tqdm
from copy import deepcopy
from mlp_numpy import MLP
from modules import CrossEntropyModule
import cifar10_utils

import torch
import logging
import matplotlib.pyplot as plt

# Accuracy and Evaluate Model

In [23]:
def accuracy(predictions, targets):
    """
    Computes the prediction accuracy, i.e., the average of correct predictions
    of the network.

    Args:
      predictions: 2D float array of size [batch_size, n_classes], predictions of the model (logits)
      targets: 1D int array of size [batch_size], ground truth labels for each sample in the batch
    Returns:
      accuracy: scalar float, the accuracy of predictions between 0 and 1,
                i.e., the average correct predictions over the whole batch
    """

    #######################
    # PUT YOUR CODE HERE  #
    #######################
    # Compute predicted classes
    predicted_class = np.argmax(predictions, axis=1)[0]
    target_class = np.argmax(targets, axis=0)
    # Calculate accuracy
    accuracy = predicted_class == target_class
    #######################
    # END OF YOUR CODE    #
    #######################

    return accuracy



def evaluate_model(model, data_loader):
    """
    Performs the evaluation of the MLP model on a given dataset.

    Args:
      model: An instance of 'MLP', the model to evaluate.
      data_loader: The data loader of the dataset to evaluate.
    Returns:
      avg_accuracy: scalar float, the average accuracy of the model on the dataset.
    """

    #######################
    # PUT YOUR CODE HERE  #
    #######################
    total_correct = 0
    total_samples = 0

    for batch_data, batch_targets in data_loader:
        batch_data = batch_data.astype(np.float32) / 255.0

        # Forward pass
        predictions = model.forward(batch_data)

        # Calculate accuracy for this batch
        batch_correct = accuracy(predictions, batch_targets)
        total_correct += batch_correct
        total_samples += batch_data.shape[0]

    # Compute average accuracy
    avg_accuracy = total_correct / total_samples
    #######################
    # END OF YOUR CODE    #
    #######################

    return avg_accuracy

# Training

In [56]:
from assignment1.train_mlp_numpy import evaluate_model
from copy import deepcopy
import numpy as np
seed = 42
data_dir = "../data"
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
hidden_dims = [128]
batch_size = 128
epochs = 2
lr = 0.1
# Set the random seeds for reproducibility
np.random.seed(seed)
torch.manual_seed(seed)

## Loading the dataset
cifar10 = cifar10_utils.get_cifar10(data_dir)
cifar10_loader = cifar10_utils.get_dataloader(cifar10, batch_size=batch_size,
                                              return_numpy=True)

#######################
# PUT YOUR CODE HERE  #
#######################

# TODO: Initialize model and loss module
# Initialize model and loss module
n_inputs = 3 * 32 * 32  # CIFAR10 images: 3 channels, 32x32 pixels
n_classes = 10
model = MLP(n_inputs, hidden_dims, n_classes)
# Print model
print(model)
loss_module = CrossEntropyModule()
# TODO: Training loop including validation
val_accuracies = []
# TODO: Test best model
test_accuracy = 0.0
# TODO: Add any information you might want to save for plotting
logging_dict = {'losses': [], 'val_accuracies': [],
                 'test_accuracy': 0.0, 'best_validation_accuracy': 0.0}

# Initialize best model
best_model = None
best_validation_accuracy = 0.0
train_acc = []
for epoch in tqdm(range(epochs), desc='Training'):
  # Training
  for batch_data, batch_targets in cifar10_loader['train']:
      # Flatten data
      batch_data = batch_data.reshape(batch_data.shape[0], -1)
      batch_data = batch_data.astype(np.float32) / 255.0  # Normalize data

      # Forward pass
      predictions = model.forward(batch_data)
      logger.debug(f"predictions.shape: {predictions.shape}")
      logger.debug(f"batch_targets.shape: {batch_targets.shape}")

      # Calculate loss
      loss = loss_module.forward(predictions, batch_targets)
      
      # Calculate accuracy 
      acc = accuracy(predictions, batch_targets)
      train_acc.append(acc)
      
      # In the training loop, after calculating loss:
      if len(logging_dict['losses']) % 100 == 0:  # Log every 100 iterations
          logger.info(f"Iteration {len(logging_dict['losses'])}, Training Loss: {loss:.4f}")
      logging_dict['losses'].append(loss)
      
      
      # Backward pass
      dL_dy = loss_module.backward(predictions, batch_targets)
      logger.debug(f"dL_dy.shape: {dL_dy.shape}")
      model.backward(dL_dy)
      
      # Update model parameters using SGD
      for layer in model.layers:
          if hasattr(layer, 'params'):
              logger.debug(f"Layer {layer}")
              # Update parameters
              for param_name, param in layer.params.items():
                  logger.debug(f"param_name: {param_name}")
                  logger.debug(f"param.shape: {param.shape}")
                  logger.debug(f"layer.grads[{param_name}].shape: {layer.grads[param_name].shape}")
                  #print(f"Initial weights", param[:5])
                  param -= lr * layer.grads[param_name]
                  #print(f"Updated weights", param[:5])

#   # Validation data
#   for val_data, val_targets in cifar10_loader['validation']:
#     # Flatten data
#     val_data = val_data.reshape(val_data.shape[0], -1)
#     val_data = val_data.astype(np.float32) / 255.0  # Normalize data
#     # One-hot encode targets
#     val_targets_onehot = np.zeros((val_targets.size, n_classes))
#     val_targets_onehot[np.arange(val_targets.size), val_targets] = 1
#     # Check shapes
#     assert val_targets_onehot.shape == (val_data.shape[0], n_classes), "Validation data shape mismatch"
#     # Evaluate model on validation set
#     val_accuracy = evaluate_model(model, zip(val_data, val_targets_onehot))
#     if len(val_accuracies) % 100 == 0:  # Log every 100 iterations
#         logger.info(f"Validation accuracy: {val_accuracy:.4f}")
#     val_accuracies.append(val_accuracy)
# 
#     if best_model is None or val_accuracy > max(val_accuracies):
#         best_validation_accuracy = val_accuracy
#         best_model = deepcopy(model)
# 
# # Evaluate best model on test set
# for test_data, test_targets in cifar10_loader['test']:
#   # Flatten data
#   test_data = test_data.reshape(test_data.shape[0], -1)
#   test_data = test_data.astype(np.float32) / 255.0  # Normalize data
#   # One-hot encode targets
#   test_targets_onehot = np.zeros((test_targets.size, n_classes))
#   test_targets_onehot[np.arange(test_targets.size), test_targets] = 1
#   # Check shapes
#   assert test_targets_onehot.shape == (test_data.shape[0], n_classes), "Test data shape mismatch"
#   # Evaluate model on test set
#   #test_accuracy = evaluate_model(best_model, zip(test_data, test_targets_onehot))
# 
# # Save logging information
# logging_dict['val_accuracies'] = val_accuracies
# logging_dict['test_accuracy'] = test_accuracy
# logging_dict['best_validation_accuracy'] = best_validation_accuracy

Files already downloaded and verified
Files already downloaded and verified



MLP with:
alpha: 1.0



Training:   0%|          | 0/2 [00:00<?, ?it/s]

2024-11-01 08:28:22,846 - INFO - Iteration 0, Training Loss: 2.3035
2024-11-01 08:28:25,262 - INFO - Iteration 100, Training Loss: 2.2969
2024-11-01 08:28:27,131 - INFO - Iteration 200, Training Loss: 2.2903
2024-11-01 08:28:28,998 - INFO - Iteration 300, Training Loss: 2.2803
2024-11-01 08:28:30,808 - INFO - Iteration 400, Training Loss: 2.2795
2024-11-01 08:28:33,193 - INFO - Iteration 500, Training Loss: 2.2660
2024-11-01 08:28:35,131 - INFO - Iteration 600, Training Loss: 2.2553
2024-11-01 08:28:36,970 - INFO - Iteration 700, Training Loss: 2.2453


In [57]:
model

In [59]:
np.mean(train_acc)

0.06552706552706553

In [32]:
for val_data, val_targets in cifar10_loader['validation']:
    # Flatten data
    val_data = val_data.reshape(val_data.shape[0], -1)
    val_data = val_data.astype(np.float32) / 255.0  # Normalize data
    batch_data = zip(val_data, val_targets_onehot)
    total_correct = 0
    total_samples = 0
    for batch_data, batch_targets in batch_data:
        batch_data = batch_data.astype(np.float32) / 255.0
    
        # Forward pass
        predictions = model.forward(batch_data)
        print(f"Predictions: {predictions}")
        #print(f"Batch targets: {batch_targets}")
        #print(predictions.shape)
        
        # Print predictions
        print(f"Prediction: {np.argmax(predictions, axis=1)[0]}")
        print(f"Target: {np.argmax(batch_targets, axis=0)}")
    
        # Calculate accuracy for this batch
        batch_correct = np.argmax(predictions, axis=1)[0] == np.argmax(batch_targets, axis=0)
        print(batch_correct)
        total_correct += batch_correct
        total_samples += batch_data.shape[0]

total_correct

Predictions: [[0.09932518 0.09999409 0.10083763 0.10035344 0.09993514 0.10282997
  0.09926299 0.09781014 0.09938503 0.10026638]]
Prediction: 5
Target: 4
False
Predictions: [[0.09925303 0.10000199 0.10083958 0.10036804 0.09996894 0.10286053
  0.09928968 0.09781995 0.09934758 0.1002507 ]]
Prediction: 5
Target: 6
False
Predictions: [[0.09929778 0.09999715 0.10083492 0.10036349 0.09994685 0.10284503
  0.09927636 0.09780878 0.09937055 0.10025908]]
Prediction: 5
Target: 1
False
Predictions: [[0.09932788 0.09998181 0.10084405 0.10037267 0.09993628 0.10284476
  0.09925734 0.09780476 0.09938875 0.10024169]]
Prediction: 5
Target: 1
False
Predictions: [[0.09931461 0.09999113 0.100845   0.10035648 0.09994319 0.10284279
  0.09926864 0.09781228 0.0993753  0.10025058]]
Prediction: 5
Target: 0
False
Predictions: [[0.09933494 0.10000547 0.10082688 0.10034495 0.09992355 0.10281978
  0.09926085 0.09779368 0.09940969 0.10028022]]
Prediction: 5
Target: 7
False
Predictions: [[0.09928121 0.09998977 0.1008477

0